<a href="https://colab.research.google.com/github/hyunseo-hy/KPI200-Index-Extraction/blob/main/JSON_crawling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [111]:
import pandas as pd
from urllib.request import urlopen, Request
import json
from pprint import pprint
from datetime import date

In [112]:
symbol = "NII@NI225"
page = 1
url = "https://finance.naver.com/world/worldDayListJson.naver?symbol=" + symbol + "&fdtc=0&page=" + str(page)

# Request 함수
# url 주소 + ID(AI 크롤링 방지 회피)
req = Request(url, headers={'User-Agent': 'Mozilla/5.0'})

try:
    # 1. 데이터 가져오기
    response = urlopen(req)
    raw_bytes = response.read()

    # 2. 디코딩 (euc-kr) 및 공백 제거(.strip())
    decoded_str = raw_bytes.decode('euc-kr').strip()

    # 3. JSON 변환
    data = json.loads(decoded_str)
    # data = json.loads(urlopen(req).read().decode('euc-kr').strip())

    # 결과 출력
    print(f"성공! 데이터 개수: {len(data)}개")
    pprint(data)
    pprint(data[0])

except Exception as e:
    print(f"에러 발생: {e}")
    if 'decoded_str' in locals():
        print("수신된 잘못된 데이터(앞부분):", decoded_str[:200])

성공! 데이터 개수: 9개
[{'clos': 52991.1,
  'diff': -592.47,
  'gvol': 126809800,
  'high': 53408.35,
  'low': 52852.9,
  'open': 53348.59,
  'rate': -1.11,
  'symb': 'NII@NI225',
  'xymd': '20260120'},
 {'clos': 53583.57,
  'diff': -352.6,
  'gvol': 122944200,
  'high': 53583.57,
  'low': 53091.45,
  'open': 53390.05,
  'rate': -0.65,
  'symb': 'NII@NI225',
  'xymd': '20260119'},
 {'clos': 53936.17,
  'diff': -174.33,
  'gvol': 144258600,
  'high': 54130.6,
  'low': 53706.79,
  'open': 54071.28,
  'rate': -0.32,
  'symb': 'NII@NI225',
  'xymd': '20260116'},
 {'clos': 54110.5,
  'diff': -230.73,
  'gvol': 150918400,
  'high': 54153.61,
  'low': 53709.87,
  'open': 54039.4,
  'rate': -0.42,
  'symb': 'NII@NI225',
  'xymd': '20260115'},
 {'clos': 54341.23,
  'diff': 792.07,
  'gvol': 144498500,
  'high': 54487.32,
  'low': 53792.68,
  'open': 53827.24,
  'rate': 1.48,
  'symb': 'NII@NI225',
  'xymd': '20260114'},
 {'clos': 53549.16,
  'diff': 1609.27,
  'gvol': 168496400,
  'high': 53814.79,
  '

In [113]:
# extract day and end price by doing so, make into a dictionary format
data[0]["xymd"] #day
data[0]["clos"] #end price

d = dict()
for n in range(len(data)):
  date = pd.to_datetime(data[n]["xymd"]).date()
  price = float(data[n]["clos"])
  d[date] = price
print(len(data))
pprint(d)

9
{datetime.date(2026, 1, 7): 51961.98,
 datetime.date(2026, 1, 8): 51117.26,
 datetime.date(2026, 1, 9): 51939.89,
 datetime.date(2026, 1, 13): 53549.16,
 datetime.date(2026, 1, 14): 54341.23,
 datetime.date(2026, 1, 15): 54110.5,
 datetime.date(2026, 1, 16): 53936.17,
 datetime.date(2026, 1, 19): 53583.57,
 datetime.date(2026, 1, 20): 52991.1}


In [114]:
# make a function

def read_json(d, symbol, page=1):
  url = "https://finance.naver.com/world/worldDayListJson.naver?symbol=" + symbol + "&fdtc=0&page=" + str(page)
  req = Request(url, headers={'User-Agent': 'Mozilla/5.0'})

  try:
      response = urlopen(req)
      raw_bytes = response.read()
      decoded_str = raw_bytes.decode('euc-kr').strip()
      data = json.loads(decoded_str)
  except Exception as e:
      print(f"에러 발생: {e}")

  for n in range(len(data)):
    date_var = pd.to_datetime(data[n]["xymd"]).date()
    price = float(data[n]["clos"])
    d[date_var] = price

  if len(data) == 10:
    page+=1
    if page >= 5: # Due to time delay, limit to the page 4
      return(d)
    read_json(d, symbol, page)
  if len(data) == 9 and str(data[0]["xymd"]) == str(date.today()):
    page+=1
    read_json(d, symbol, page)

  return(d)

In [115]:
symbol = "NII@NI225"
page = 1
historical_idx = pd.Series()
historical_idx = read_json(historical_idx, symbol, page)
historical_idx.head(3)

,0
2026-01-20,52991.10
2026-01-19,53583.57
2026-01-16,53936.17


In [116]:
indices = {
    "SPI@SPX" : "S&P 500",
    "NAS@NDX" : "Nasdaq 100",
    "NII@NI225" : "Nikkei 225"
}

historical_indices = dict()
for key, value in indices.items():
  print(key, value)
  s = dict()
  s = read_json(s, key, 1)
  historical_indices[value] = s
price_df = pd.DataFrame(historical_indices)
price_df = price_df.sort_index(ascending=False)
price_df.head(3)

SPI@SPX S&P 500
NAS@NDX Nasdaq 100
NII@NI225 Nikkei 225


,S&P 500,Nasdaq 100,Nikkei 225
2026-01-20,6796.86,24987.57,52991.10
2026-01-19,NaN,NaN,53583.57
2026-01-16,6940.01,25529.26,53936.17


In [117]:
def date_format(d=""):
  if d != "":
    this_date = pd.to_datetime(d).date()
  else:
    this_date = pd.Timestamp.today().date()
  return(this_date)

In [118]:
def idx_global(d, symbol, start_date="", end_date=date.today(), page=1):
  if start_date == "":
    start_date = end_date - pd.DateOffset(months=1) # if user doesn't assign a start date then, assign as a month before the end date (default setting)
  start_date = date_format(start_date)

  url = "https://finance.naver.com/world/worldDayListJson.naver?symbol=" + symbol + "&fdtc=0&page=" + str(page)
  req = Request(url, headers={'User-Agent': 'Mozilla/5.0'})
  try:
    response = urlopen(req)
    raw_bytes = response.read()
    decoded_str = raw_bytes.decode('euc-kr').strip()
    data = json.loads(decoded_str)
  except Exception as e:
    print(f"에러 발생: {e}")

  if len(data) > 0:

    for n in range(len(data)):
      date = pd.to_datetime(data[n]["xymd"]).date()
      if date <= end_date and date >= start_date:
        d[date] = price = float(data[n]["clos"])
  if len(data) == 9 and str(data[0]["xymd"]) == str(date.today()):
    page+=1
    idx_global(d,symbol, start_date, end_date, page)

  return(d)

In [122]:
indices = {
    "SPI@SPX" : "S&P 500",
    "NAS@NDX" : "Nasdaq 100",
    "NII@NI225" : "Nikkei 225"
}

historical_indicies = dict()
start_date = "2026-1-1"
end_date = "2026-1-30"
for key, value in indices.items():
  s = dict()
  s = idx_global(s,key,start_date)
  historical_indices[value] = s
prices_df = pd.DataFrame(historical_indices)
prices_df.sort_index(ascending=False)

2026-01-01 2026-01-21
2026-01-01 2026-01-21
2026-01-01 2026-01-21


,S&P 500,Nasdaq 100,Nikkei 225
2026-01-20,6796.86,24987.57,52991.10
2026-01-19,NaN,NaN,53583.57
2026-01-16,6940.01,25529.26,53936.17
2026-01-15,6944.47,25547.08,54110.50
2026-01-14,6926.60,25465.94,54341.23
2026-01-13,6963.74,25741.95,53549.16
2026-01-12,6977.27,25787.66,NaN
2026-01-09,6966.28,25766.26,51939.89
2026-01-08,6921.46,25507.10,51117.26
2026-01-07,6920.93,25653.90,51961.98
